<a href="https://colab.research.google.com/github/LaTsa99/DeepMinds/blob/master/KerasTunerTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-gpu
!pip install keras-tuner

In [ ]:
import os, zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

!pip install gdown
!gdown https://drive.google.com/uc?id=1UM65-NZoXAr3lydSkmX2OmHm63-PfepX -O /tmp/girls.zip

local_zip = '/tmp/girls.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = os.path.join('/tmp', 'girls')

train_dir = os.path.join(base_dir, 'train')
train_bikini_dir = os.path.join(train_dir, 'bikini')
train_long_dir = os.path.join(train_dir, 'long')

valid_dir = os.path.join(base_dir, 'valid')
valid_bikini_dir = os.path.join(valid_dir, 'bikini')
valid_long_dir = os.path.join(valid_dir, 'long')

test_dir = os.path.join(base_dir, 'test')
test_bikini_dir = os.path.join(test_dir, 'bikini')
test_long_dir = os.path.join(test_dir, 'long')

bikini_train_fnames = os.listdir(train_bikini_dir)
bikini_valid_fnames = os.listdir(valid_bikini_dir)
bikini_test_fnames = os.listdir(test_bikini_dir)

long_train_fnames = os.listdir(train_long_dir)
long_valid_fnames = os.listdir(valid_long_dir)
long_test_fnames = os.listdir(test_long_dir)

img_height = 299
img_width = 299

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=10, zoom_range=0.2, horizontal_flip=True, rotation_range=20)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(img_height, img_width), batch_size=20, class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(img_height, img_width), batch_size=20, class_mode='binary')
test_generator  = test_datagen.flow_from_directory(test_dir, target_size=(img_height, img_width), batch_size=20, class_mode='binary')

In [30]:
early_stopping = EarlyStopping(patience=10, verbose=1, min_delta=0.001)               # Early stopping to stop overfitting
checkpointer = ModelCheckpoint(filepath='model.hdf5', save_best_only=True, verbose=1) # Checkpointer to save the best model
tb = TensorBoard(log_dir='logs', histogram_freq=1, write_graph=1) 

In [28]:
from tensorflow.keras.layers import Dense, GlobalMaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.optimizers import Adam

def build_model(hp):
  base_model = InceptionV3(weights='imagenet', include_top=False)

  x = base_model.output
  x = GlobalMaxPooling2D()(x)
  x = Dropout(
      rate=hp.Choice('dropout_1', values=[0.0, 0.1, 0.15, 0.25, 0.3, 0.45, 0.5])  # finomít
  )(x)
  x = Dense(1024,
      activation=hp.Choice('activation', ['swish', 'relu', 'leakyrelu', 'sigmoid'])
  )(x)
  x = Dropout(
      rate=hp.Choice('dropout_2', values=[0.0, 0.1, 0.15, 0.25, 0.3, 0.45, 0.5])  # finomít
  )(x)
  predictions = Dense(1, activation='sigmoid')(x)

  model = Model(inputs=base_model.input, outputs=predictions)

  for layer in base_model.layers:
    layer.trainable = False

  model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam(hp.Choice('learning_rate', values=[1e-2,1e-3])))
  return model

In [29]:
from kerastuner.tuners import Hyperband

tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    factor=3,
    max_epochs=5,
    directory='output',
    project_name='hyper1'
)

tuner.search_space_summary()

INFO:tensorflow:Reloading Oracle from existing project output/hyper1/oracle.json
Search space summary
Default search space size: 5
dropout_1 (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.25, 0.5], 'ordered': True}
dense_1_units (Choice)
{'default': 256, 'conditions': [], 'values': [256, 512, 1024], 'ordered': True}
activation (Choice)
{'default': 'swish', 'conditions': [], 'values': ['swish', 'relu', 'leakyrelu', 'sigmoid'], 'ordered': False}
dropout_2 (Choice)
{'default': 0.0, 'conditions': [], 'values': [0.0, 0.25, 0.5], 'ordered': True}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001], 'ordered': True}


In [31]:
tuner.search(train_generator, steps_per_epoch=79, validation_data=valid_generator, validation_steps=26, epochs=10, callbacks=[early_stopping, checkpointer, tb])

Trial 10 Complete [00h 04m 01s]
val_accuracy: 0.9692307710647583

Best val_accuracy So Far: 0.9730769395828247
Total elapsed time: 00h 25m 37s
INFO:tensorflow:Oracle triggered exit


In [34]:
params_best = tuner.get_best_hyperparameters(num_trials=1)[0]
params_best.get_config()['values']

{'activation': 'relu',
 'dense_1_units': 256,
 'dropout_1': 0.25,
 'dropout_2': 0.5,
 'learning_rate': 0.01,
 'tuner/bracket': 1,
 'tuner/epochs': 2,
 'tuner/initial_epoch': 0,
 'tuner/round': 0}

In [ ]:
model_best = tuner.hypermodel.build(params_best)
print(model_best.summary())